# SIFT (Lowe)

- Uses Difference of Gaussian (DoG) in both space and scale

## 1. Constructing a Scale Space: 

This is the initial preparation. You create internal representations of the original image to ensure scale invariance. This is done by generating a "scale space".
- Mathematically, "blurring" (or smoothing) is referred to as **the convolution of the Gaussian operator and the image**.
- Typically used to reduce image noise and reduce detail.
- $\sigma$ (standard deviation) is the **"scale" parameter**. Think of it as the amount of blur. **Greater the value, greater the blur.**
- Since the Fourier transform of a Gaussian is another Gaussian, applying a Gaussian blur has the effect of reducing the image's high-frequency components; a Gaussian blur is thus a low pass filter. 
- Gaussian operator is a separable filter. That is, the effect of applying the two-dimensional matrix can also be achieved by applying a series of single-dimensional Gaussian matrices in the horizontal direction, then repeating the process in the vertical direction. In computational terms, this is a useful property, since the calculation can be performed in $O(w_{ker}w_{im}h_{im})+O(h_{ker}w_{im}h_{im})$ as opposed to $O(w_{ker}h_{ker}w_{im}h_{im})$ for a non-separable kernel.

### Scale Space in General:

- Do you want to look at a leaf or the entire tree? If it's a tree, get rid of some detail from the image (like the leaves, twigs, etc) intentionally.
- While getting rid of these details, you must ensure that you do not introduce new false details. The only way to do that is with the Gaussian Blur (it was proved mathematically, under several reasonable assumptions).
- So to create a scale space, you take the original image and generate progressively blurred out images.

### Scale Spaces in SIFT: 

- You take the original image, and generate progressively blurred out images. 
- Then, you resize the original image to half size. 
- And you generate blurred out images again.

## 2. LoG Approximation: 

The Laplacian of Gaussian is great for finding interesting points (or key points) in an image. But it's computationally expensive. So we cheat and approximate it using the representation created earlier.

### Laplacian of Gaussian:

- The Laplacian of Gaussian (LoG) operation works as follows: 
  - You take an image, and blur it a little. 
  - And then, you calculate second order derivatives on it (or, the "laplacian"). 
  - This locates edges and corners on the image. These edges and corners are good for finding keypoints.
- Normally, the second order derivative is extremely sensitive to noise. The blur smoothes out the noise and stabilizes the second order derivative.
- The problem is, calculating all those second order derivatives is computationally intensive.

### Difference of Gaussians:

![image](sift-dog-idea.jpg)

- To generate Laplacian of Guassian images quickly and efficiently, we use the scale space. We calculate the difference between two consecutive scales, or the Difference of Gaussians.
- These Difference of Gaussian images are approximately equivalent to the Laplacian of Gaussian.
- These approximations are also "scale invariant".

### LoG vs DoG:

- The Laplacian of Gaussian images are not alone scale invariant (i.e. they depend on the amount of applied blur) due to the Gaussian expression.
- The $\sigma^2$ in the denominator (owing to taking derivates) in the LoG is the scale. If we get rid of it by multiplying with $\sigma^2$, we'll have true scale independence.
- But the resultant images after the DoG operation are already multiplied by the $\sigma^2$. So, the requirement is taken care of by the Difference of Gaussian operation.

### Side Effects of DoG:

- The DoG result is multiplied with k-1 as well. (This is the k in the expression k*$\sigma$.)
- But we'll just be looking for the location of the maximums and minimums in the images. We'll never check the actual values at those locations. So, this additional factor won't be a problem to us because even if you multiply throughout by some constant, the maxima and minima stay at the same location.

## 3. Finding keypoints: 

With the fast approximation, we now try to find key points. These are maxima and minima in the Difference of Gaussian image we calculate in step 2

Finding key points is a two part process:  
1. Locate maxima/minima in DoG images  
2. Find subpixel maxima/minima

### 1. Locate Maxima/Minima in DoG Images:
- The first step is to roughly locate the maxima and minima. You simply iterate through each pixel and check all it's neighbours. The check is done within the current image, and also the one above and below it. Something like this:
![image.png](sift-maxima-idea.jpg)
- X marks the current pixel. The green circles mark the neighbours. This way, a total of 26 checks are made. X is marked as a "key point" if it is the greatest or least of all 26 neighbours.
- Usually, a non-maxima or non-minima position won't have to go through all 26 checks. A few initial checks will usually sufficient to discard it.
- Note that keypoints are not detected in the lowermost and topmost scales. There simply aren't enough neighbours to do the comparison. So simply skip them!
- Once this is done, the marked points are the approximate maxima and minima. They are "approximate" because the maxima/minima almost never lies exactly on a pixel. It lies somewhere between the pixel and we simply cannot access data "between" pixels. So, we must mathematically locate the subpixel location.
![image.png](sift-maxima-subpixel.jpg)  
<p style="text-align: center;"> Visual explanation: The red crosses mark pixels in the image but the actual extreme point is the green one.</p>

### 2. Find Subpixel Maxima/Minima:

- Using the available pixel data, subpixel values are generated. 
- This is done by the Taylor expansion of the image around the approximate key point.
- We can easily find the extreme points of the expansion expression (differentiate and equate to zero). 
- On solving, we'll get subpixel key point locations. These subpixel values increase chances of matching and stability of the algorithm.

#### Note: 

To generate, say, two extrema (maxima/minima) images (like the example below): 
- We need exactly 4 DoG images. 
- To generate 4 DoG images, you need 5 Gaussian blurred images (5 level of blurs).

![image](sift-maxima-detector.jpg)

## 4. Get Rid of Bad Key Points:

Edges and low contrast regions are bad keypoints. Eliminating these makes the algorithm efficient and robust. A technique similar to the Harris Corner Detector is used here.

### Removing Low Contrast Features:

- If the magnitude of the intensity (i.e., without sign) at the current pixel in the DoG image (that is being checked for minima/maxima) is less than a certain value, it is rejected.
- Note that because we have subpixel keypoints (we used the Taylor expansion to refine keypoints), we again need to use the Taylor expansion to get the intensity value at subpixel locations.

### Removing Edges:

- The idea is to calculate two gradients at the keypoint, both of which are perpendicular to each other. 
- The image around the keypoint can be:
  - **A flat region:** If this is the case, both gradients will be small.
  - **An edge:** Here, one gradient will be big (perpendicular to the edge) and the other will be small (along the edge)
  - **A corner:** Here, both gradients will be big.
- Corners are great keypoints. So, we want just corners. If both gradients are big enough, we let it pass as a key point. Otherwise, it is rejected.
- Mathematically, this is achieved by the Hessian Matrix. (Almost the same math foundation as the Harris Corner Detector is used in SIFT.) Using this matrix, you can easily check if a point is a corner or not. [See Hessian Matrix on Wikipedia](https://en.wikipedia.org/wiki/Hessian_matrix)

## 5. Assigning an Orientation to the Keypoints:

We already know the scale at which the keypoint was detected (it's the same as the scale of the blurred image). So we have scale invariance. Next, an orientation is calculated for each keypoint. Any further calculations are done relative to this orientation. This effectively cancels out the effect of orientation, making it rotation invariant.

### Finding the Overall Orientation for a Region:

- The idea is to collect gradient directions/orientations (i.e. angles) and magnitudes **around** each keypoint. 
- Then we figure out the most prominent orientation(s) in that region (by using histograms). 
- And we assign this orientation(s) to the keypoint.
- Any later calculations are done relative to this orientation. This ensures rotation invariance.

#### Note: 

The size of the "orientation collection region" around the keypoint depends on it's scale. **The bigger the scale, the bigger the collection region.**

### Generate Orientation Histogram for a Keypoint:

To assign an orientation we use a histogram and a small region around it. Using the histogram, the most prominent gradient orientation(s) are identified as follows:
- The magnitude and orientation is calculated for all pixels around the keypoint. Then, a histogram is created for this.
- In this histogram, the 360 degrees of orientation are divided into 36 bins (each 10°).
- Once you've done this for all pixels around the keypoint, the histogram will have a peak at some point:
  - If there is only one peak, it is assigned to the keypoint. (i.e. no other peak has the same frequency or a frequency higher than 80% of the frequency of this peak.)
  - If there are multiple peaks above the 80% mark, they are all converted into a new keypoint (with their respective orientations).
  
![image](sift-orientation-histogram.jpg)

- For example, the above histogram peaks at 20-29 degrees. So, the keypoint is assigned to the orientation 3 (the third bin).
- Also, any peaks above 80% of the highest peak are converted into a new keypoint. This new keypoint has the same location and scale as the original. But it's orientation is equal to the other peak.
- So, orientation can split up one keypoint into multiple keypoints.
- To become rotation invariant, rotate each gradient direction and location by the negative of keypoint's orientation!

## 6. Generate SIFT Features:

Finally, with scale and rotation invariance in place, one more representation is generated. Now, we create a fingerprint for each keypoint. This helps uniquely identify keypoints. If an eye is a keypoint, then using its fingerprint, we'll be able to distinguish it from other keypoints, like ears, noses, fingers, etc.

### Generate a Unique Fingerprint for the Keypoint:

Since things are never exactly the same when comparing two different images, as well as making them unique and easy to compute, we want fingerprints to be relatively fault-tolerant when it is being compared against other keypoints:
- Create a 16x16 window out of the region defined around the keypoint in the previous step. This 16x16 window is divided into sixteen 4x4 windows. **(i.e. 16 window each of which is of size 16 pixels.)**

![image](sift-fingerprint.jpg)

- Within each 4x4 window, gradient magnitudes and orientations are calculated. These orientations are put into an 8 bin histogram. **(each 45°, a histogram of 8 bins will be generated out of each window.)**

![image](sift-4x4.jpg)

- As an example, orientation in the range 0-44 degrees falls into the first bin, the range 45-89 falls into the next bin and so on.
- The amount added to the bin depends on the magnitude of the gradient. Unlike the past, the amount added also depends on the distance from the keypoint. So, gradients that are far away from the keypoint will add smaller values to the histogram (i.e. magnitude value is weighted inversely proportional with respect to the distance, this can be done by using a "Gaussian weighting function".).
- After those steps:
  - There are **16 histograms** for sixteen 4x4 windows around each keypoint.
  - Each generated histogram consists of **8 bins**.
  - As a result, the feature vector for a keypoint is of dimension **16*8=128**.
- Once you have all 128 numbers, you normalize them (divide by root of sum of squares). These 128 numbers form the "feature vector". This keypoint is uniquely identified by this feature vector.

#### Note:

Most of the time, the keypoint does not lie exactly on a pixel but it is situated between pixels. The 16x16 window takes orientations and magnitudes of the image "in-between" pixels. So, you need to interpolate the image to generate orientation and magnitude data "in between" pixels.

### Notes on Rotation and Illumination Invariant  Feature Vector:

#### Rotation dependence: 

Rotation independence is achieved by rotating each gradient direction and location by the negative of keypoint's orientation **before the 6th step**. This allows expressing the orientation and location of each gradient with respect to the keypoint's orientation.

#### Illumination dependence:

If we threshold numbers that are big, we can achieve illumination independence. **So, any number (of the 128) greater than 0.2 is clamped to 0.2.** This resultant feature vector is normalized again. Now, this yields illumination independent feature vector.